# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

In [ ]:
ls

**Original source folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing**

**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

inputName = "person"
drivingVideo = "damefinal.gif"

source_image = imageio.imread(f"./inputs/{inputName}.png")
driving_video = imageio.mimread(f"./inputs/{drivingVideo}")


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
print("Done!")
# HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./vox-cpk.pth.tar')
print("Done!")

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte
import os

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave(f"./outputs/{inputName}.mp4", [img_as_ubyte(frame) for frame in predictions], fps=30)
os.system(f"ffmpeg -i outputs/{inputName}.mp4 -i inputs/damedane.mp4 -vcodec copy -acodec copy final/{inputName}.mp4")
print("Video Complete!")


HTML(display(source_image, driving_video, predictions).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proportions will be inherited from the driving video.**

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

**One Click One Deepfake :)**

In [ ]:
#Configuration

inputName = "person"
drivingVideo = "damefinal.gif"
audioSource = "damedane.mp4"

print(f"Ready to go for input  {inputName}.png\n with driving video    {drivingVideo}\n and audio source      {audioSource}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from demo import load_checkpoints, make_animation
from skimage import img_as_ubyte
import os, warnings, imageio
warnings.filterwarnings("ignore")
source_image = imageio.imread(f"./inputs/{inputName}.png")
driving_video = imageio.mimread(f"./inputs/{drivingVideo}")
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='./vox-cpk.pth.tar')
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
if (os.path.exists(f"./outputs/{inputName}.mp4")): os.remove(f"./outputs/{inputName}.mp4")
imageio.mimsave(f"./outputs/{inputName}.mp4", [img_as_ubyte(frame) for frame in predictions], fps=30)
os.system(f"ffmpeg -i outputs/{inputName}.mp4 -i inputs/{audioSource} -vcodec copy -acodec copy -y final/{inputName}.mp4")
print("Video Complete!")
# HTML(display(source_image, driving_video, predictions).to_html5_video())